In [89]:
import pandas as pd
import re
import numpy as np
from pyspark import SparkContext
from pyspark.sql import Row, SparkSession
import time
import csv


# spark_context = SparkSession.builder.master("local").appName("main").getOrCreate()

fire_dep = pd.read_csv('Seattle_Real_Time_Fire_911_Calls.csv')
contractors = pd.read_csv('L_I_Contractor_License_Data_-_General.csv')
abbr = pd.read_csv('street_abbr.csv')
# wa_voters = pd.read_csv('WashingtonStateVoterExtract/20221101_VRDB_Extract.txt', sep = '|')

In [92]:
vars_to_use = [('FName', 1), ('MName', 2), ('LName', 3), ('RegStNum', 7), ('RegStName', 9), ('RegStType', 10), ('RegCity', 15), ('Registrationdate', 31)]
# Registration Date and the combintion of RegStNum, RegStName, and RegStFrac are important for the join. 

voters = []

voters_path = 'WashingtonStateVoterExtract/20221101_VRDB_Extract.txt'
with open(voters_path) as file:
    wa_voters = csv.reader(file, delimiter="|")
    for row in wa_voters:
        voter = {}
        for var_index in vars_to_use:
            voter.update({var_index[0]: row[var_index[1]]})
        voters.append(voter)
        

In [93]:
voters_df = pd.DataFrame(voters[1:len(voters)], columns = voters[0])

In [95]:
abbr['full'] = abbr['full'].str.lower()
abbr['abbr'] = abbr['abbr'].str.lower()
abbr.head()

,full,abbr
0,alley,aly
1,avenue,ave
2,boulevard,blvd
3,causeway,cswy
4,center,ctr


In [96]:
# Need to data clean with the steps below:
# lower
# remove punctuation

def apply_regex(address):
    '''Uses regular expressions to format the address data fora join.'''
    
    address = str(address)
    
    new_str = address.lower()
    end_str = re.sub(r'[^\w\s]', '', new_str)
    
    for i in range(len(abbr)):
        end_str = end_str.replace(abbr['full'][i], abbr['abbr'][i])
    
    return end_str

print(apply_regex(contractors['Address1'][0]))

print("""Expected Runtime is ~9min. \nOver 1 mil records and expensive str search operation.\n
    Could parallelize with Spark but that's too much work""")

start = time.perf_counter()

contractors['join_address'] = contractors['Address1'].apply(apply_regex)
fire_dep['join_address'] = fire_dep['Address'].apply(apply_regex)
print("Time: ", start - time.perf_counter())


11506 meridian ave n
Expected Runtime is ~9min. 
Over 1 mil records and expensive str search operation.

    Could parallelize with Spark but that's too much work
Time:  -565.6257715000029


In [97]:
# Should take about 10 minutes to run
voters_df['concated'] = voters_df['RegStNum'] + voters_df['RegStName'] + voters_df['RegStType']

voters_df['address'] = voters_df['concated'].apply(apply_regex)


In [98]:
# Need to get only the most recent contractor for each address in the contractors dataset. 
contractors['recency'] = contractors.sort_values(['LicenseEffectiveDate'])\
    .groupby(['join_address'])\
    .cumcount() + 1

# a recency of 1 is the newest record. 

contractors[contractors['join_address'] == '0607 sw idaho st'][['LicenseEffectiveDate', 'recency']]

contractors = contractors[contractors['recency'] == 1]

In [99]:
# Need to think about other data cleaning steps to improve the re-identification. 

In [110]:
# Manages the join. 

joined_data = pd.merge(fire_dep, contractors, on = "join_address", how = 'left')
joined_data.sort_values(['recency'])
print('hiding head of data from Github for privacy')

hiding head of data from Github for privacy


In [101]:
# Calculates what percentage of fire department emergencies are linked to contractors. 

total_incidents = len(joined_data)
joined_incidents = len(joined_data[np.isnan(joined_data['recency']) == False])


In [102]:
print('Percent of the dataset identified: ', 100 * joined_incidents / total_incidents)

Percent of the dataset identified:  1.225101952372791


In [103]:
# need to think about joining on dates so that you only get incidents 
# that are after the contractor moved into the unit. 

In [66]:
joined_incidents

21230

In [105]:
voters_df['join_address'] = voters_df['address']
joined_data_full = pd.merge(joined_data, voters_df, on = "join_address", how = 'left')

In [108]:
joined_data_full.to_csv('joined_data_full.csv')